### Imports

In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Import Pixar csv

In [16]:
pixar_df = pd.read_csv('../data/pixar__studio12.csv')
pixar_df.sample(5)

,Moviename,Releaseyear,Budget(million),US and canada(million),Other territories(million),Worldwide(million),Rotten Tomatoes
16,Finding Dory,2016,2000,4863,5423,10286,94%
9,Up,2009,1750,2930,4421,7351,98%
1,A Bug's Life,1998,1200,1628,2005,3633,92%
21,Onward,2020,2000,616,804,1420,88%
6,Cars,2006,1200,2441,2179,4620,74%


## Scrapping wiki table

In [17]:
wiki = 'https://en.wikipedia.org/wiki/List_of_Pixar_films'

In [18]:
res = requests.get(wiki)
soup = BeautifulSoup(res.content, "html.parser")

In [19]:
wrappers = soup.find_all('table', {'class':'wikitable'})
wrappers[4]

<table class="wikitable sortable" style="text-align:center; margin=auto; font-size:90%">
<tbody><tr>
<th rowspan="2" scope="col">Film
</th>
<th rowspan="2" scope="col"><a href="/wiki/Academy_Award_for_Best_Picture" title="Academy Award for Best Picture">Best Picture</a>
</th>
<th rowspan="2" scope="col"><a href="/wiki/Academy_Award_for_Best_Animated_Feature" title="Academy Award for Best Animated Feature">Animated Feature</a>
</th>
<th rowspan="2" scope="col"><a href="/wiki/Academy_Award_for_Best_Original_Screenplay" title="Academy Award for Best Original Screenplay">Original Screenplay</a>
</th>
<th rowspan="2" scope="col"><a href="/wiki/Academy_Award_for_Best_Adapted_Screenplay" title="Academy Award for Best Adapted Screenplay">Adapted Screenplay</a>
</th>
<th rowspan="2" scope="col"><a href="/wiki/Academy_Award_for_Best_Original_Score" title="Academy Award for Best Original Score">Original Score</a>
</th>
<th rowspan="2" scope="col"><a href="/wiki/Academy_Award_for_Best_Original_Son

### Academy Awards Table

    Column names extraction

In [20]:
col_names = wrappers[4].find_all('th')
col_names = [i.getText().strip() for i in col_names]
col_names.remove('Sound[a]')
col_names = [i.lower().replace(' ', '_') for i in col_names]
col_names.remove('other')
col_names.append('other')
col_names

['film',
 'best_picture',
 'animated_feature',
 'original_screenplay',
 'adapted_screenplay',
 'original_score',
 'original_song',
 'sound_editing',
 'sound_mixing',
 'other']

    Rows extraction

In [21]:
def extract_wiki_table(rows, columns):
    # dictionary to control rowspan values in each iteration
    rowspan_values = {col: 1 for col in columns}
    data = []

    for idx, row in enumerate(rows):
        # list_ will be the list of values for each row in the table
        list_ = []
        cells = row.find_all('td')
        count_cells = 0
        for i in range(len(columns)):
            # check if last_row[column] rowspan is greater than 1 to pick the value of that cell
            # update rowspan of this row decreasing in 1
            if int(rowspan_values[columns[i]]) > 1:
                last_row = data[idx - 1]
                last_value = last_row[i]
                list_.append(last_value)
                rowspan_values[columns[i]] -= 1
            else:
                list_.append(cells[count_cells].getText().strip().replace('\\n', ''))
                if cells[count_cells].get('rowspan'):
                    rowspan_values[columns[i]] = int(cells[count_cells].get('rowspan'))
                else:
                    rowspan_values[columns[i]] = 1
                count_cells += 1
        data.append(list_)
    
    return pd.DataFrame(data, columns=columns)

In [22]:
rows = wrappers[4].find_all('tr')
rows = rows[2::]

# Excluding 4 last rows to extract the info
rows = rows[:22]
df_1 = extract_wiki_table(rows, col_names)
df_1.sample(3)

,film,best_picture,animated_feature,original_screenplay,adapted_screenplay,original_score,original_song,sound_editing,sound_mixing,other
13,Monsters University,,,Ineligible,,,,,,
0,Toy Story,,Award not yet introduced,Nominated,Ineligible,Nominated,Nominated,,,Won Special Achievement
14,Inside Out,,Won,Nominated,Ineligible,,,,,


    The last 4 movies in the table from wikipedia were added by hand because there was a problem with the columns, which did not match.

In [23]:
remaining_movies = [
    ['Soul', '', 'Won', '', 'Ineligible', 'Won', '', 'Nominated', 'Nominated', ''],
    ['Luca', '', 'Nominated', '', 'Ineligible', '', '', '', '', ''],
    ['Turning Red', '', 'Nominated', '', 'Ineligible', '', '', '', '', ''],
    ['Lightyear', '', '', 'Ineligible', '', '', '', '', '', '']
]
df_2 = pd.DataFrame(remaining_movies, columns=col_names)
df_2

,film,best_picture,animated_feature,original_screenplay,adapted_screenplay,original_score,original_song,sound_editing,sound_mixing,other
0,Soul,,Won,,Ineligible,Won,,Nominated,Nominated,
1,Luca,,Nominated,,Ineligible,,,,,
2,Turning Red,,Nominated,,Ineligible,,,,,
3,Lightyear,,,Ineligible,,,,,,


    The two data frames were then concatenated to merge them into one.

In [24]:
academy_awards_df = pd.concat([df_1, df_2], ignore_index=True)

### Box Office Table

    Column names extraction

In [25]:
col_names = wrappers[2].find_all('th')
col_names = [i.getText().strip() for i in col_names]
col_names.remove('Box office gross')
col_names = [i.lower().replace(' ', '_') for i in col_names]
col_names.remove('ref.')
col_names.append('ref.')
col_names

['year',
 'film',
 'budget',
 'u.s._and_canada',
 'other_territories',
 'worldwide',
 'ref.']

    Rows extraction

In [26]:
rows = wrappers[2].find_all('tr')
rows = rows[2::]
df = extract_wiki_table(rows, col_names)
df = df.drop(columns=["ref."])
df

,year,film,budget,u.s._and_canada,other_territories,worldwide
0,1995,Toy Story,$30 million,"$192,523,233","$172,747,718","$365,270,951"
1,1998,A Bug's Life,$120 million,"$162,798,565","$200,460,294","$363,258,859"
2,1999,Toy Story 2,$90 million,"$245,852,179","$265,506,097","$511,358,276"
3,2001,"Monsters, Inc.",$115 million,"$289,916,256","$342,400,393","$632,316,649"
4,2003,Finding Nemo,$94 million,"$339,714,978","$531,300,000","$871,014,978"
5,2004,The Incredibles,$92 million,"$261,441,092","$370,165,621","$631,606,713"
6,2006,Cars,$120 million,"$244,082,982","$217,900,167","$461,983,149"
7,2007,Ratatouille,$150 million,"$206,445,654","$417,280,431","$623,726,085"
8,2008,WALL-E,$180 million,"$223,808,164","$297,503,696","$521,311,860"
9,2009,Up,$175 million,"$293,004,164","$442,094,918","$735,099,082"


In [27]:
final_df = pd.merge(academy_awards_df, df, on="film", how="outer")
final_df.to_csv('../data/pixar_academy_awards.csv', index=False)

In [28]:
def create_dataframe_from_rows_and_columns(rows, columns):
    # dictionary to control rowspan values in each iteration
    rowspan_values = {col: 1 for col in columns}
    colspan = 1
    data = []

    for idx, row in enumerate(rows):
        # list_ will be the list of values for each row in the table
        list_ = []
        cells = row.find_all('td')
        count_cells = 0
        for i in range(len(columns)):
            # check if last_row[column] rowspan is greater than 1 to pick the value of that cell
            # update rowspan of this row decreasing in 1
            if int(rowspan_values[columns[i]]) > 1:
                last_row = data[idx - 1]
                print(columns[i])
                print(last_row)
                last_value = last_row[i]
                list_.append(last_value)
                rowspan_values[columns[i]] -= 1
            elif colspan > 1:
                list_.append(list_[i - 1])
                colspan -= 1
                count_cells -= 1
            else:
                list_.append(cells[count_cells].getText().strip().replace('\\n', ''))
                if cells[count_cells].get('rowspan') and cells[count_cells].get('colspan'):
                    rowspan_values[columns[i]] = int(cells[count_cells].get('rowspan'))
                    count_cells += 1

                else:
                    rowspan_values[columns[i]] = 1
                    count_cells += 1

            if cells[count_cells].get('colspan'):
                colspan = cells[count_cells].get('colspan')
                count_cells -= 1
            else: 
                colspan = 1
        data.append(list_)
    print(rowspan_values)
    print(data)
    
    return pd.DataFrame(data, columns=columns)

rows = wrappers[4].find_all('tr')
rows = rows[2::]
rows = rows[:22]
df = create_dataframe_from_rows_and_columns(rows, col_names)
df

IndexError: list index out of range